## Visualização e pré-processamento

A seguir, a implementação de cada um dos filtros: passa-faixa (bandpass), passa-baixa (lowpass), passa-alta (highpass) e corta-faixa (notch).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from sklearn.preprocessing import LabelEncoder
import mne


# definições de filtros

# filtro em que você deseja extrair frequências em uma faixa específica de interesse
def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)

# filtro usado quando você deseja remover componentes de alta frequência do sinal
def butter_lowpass(data, lowcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)

# filtro usado para remover componentes de baixa frequência do sinal
def butter_highpass(data, highcut, fs=512, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)

# filtro para remover interferências de frequência, como ruído elétrico de 50 ou 60 Hz em dados EEG.
def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

Para imprimir os dados no domínio da frequência, foi utilizado a função `matplotlib.pyplot.psd` que na verdade utiliza a transformação dos dados por FFT, porém aplica uma característica que evidencia a "energia" dos dados, chamado de PSD (_Power Spectrum Density_)

Definição de função para imprimir os dados no domínio do tempo e da frequência:

In [ ]:
# psd é pegar todos os dados transformados no domínio do tempo e frequência, deixando mais evidente as informações com a amplitude (potência mais alta)
# quando coloca no domínio da frequência vem a parte imaginária, por isso faz módulo. psd consegue mostrar a parte imaginária. serve para a classificação
# cores -> energia do sinal 
def print_graphs(data):
    for i in range(0, 21, 3):
        plt.plot(data[i,:])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(0, 21, 3):
        plt.psd(data[i,:], Fs=512)
    plt.title('Domínio da frequência')
    plt.show()

    for i in range(0, 21, 3):
        plt.specgram(data[i,:], Fs=512)
    plt.title('Espectograma')
    plt.show()

Carregamento dos dados de um participante da base de dados AVI:

In [ ]:
data = np.load("datasets/avi/single/data_sub1.npy")
print(data.shape)

# Transpondo dados para organizar os dados na última dimensão
data = data.T
data.shape

Visualização do sinal em todos os domínios apresentados, sem a aplicação de nenhum filtro:

In [ ]:
# definição dos tamanhos das figuras
rcParams['figure.figsize'] = [16., 5.]
print_graphs(data)

Para facilitar o trabalho de visualização dos dados nas faixas de frequência de trabalho, vamos utilizar a ferramenta `MNE`:

In [ ]:
#abcissa = frequencias
# criação de um objeto "info"
n_channels = 1 # criação de um canal. no beta vai ter mais
sfreq = 512 #64 beta; 512 amostras serão registradas pors egundo (2x256)
ch_names = ['Oz'] # nome do canal
ch_types = ['eeg'] # tipo do canal
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
info

In [ ]:
from sklearn.preprocessing import LabelEncoder

labels = np.load("datasets/avi/single/labels_sub1.npy")[0]
print(labels, labels.shape)
#repete os targets 3 vezes

In [ ]:
# data : array, shape (n_epochs, n_channels, n_times)
data_correct = data.reshape(data.shape[0], 1, data.shape[1])
# adicionando label como "épocas" do sinal
labels = np.load("datasets/avi/single/labels_sub1.npy")[0]
# método para transformar labels categóricos
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(21, dtype=int),
    le.fit_transform(labels))
)
print(events) # a coluna do meio cortou nada, a última coluna é identificador do evento
event_dict = {'6':0, '6.5':1, '7':2, '7.5':3, '8.2':4, '9.3':5, '10':6} # 6Hz está armazenado em 0...
mne_data = mne.EpochsArray(data_correct, info, events, event_id=event_dict) # 3 eventos com 10...
mne_data

In [ ]:
#ocorreram 21 evocações
# view = mne_data.compute_psd(method='welch', fmin=3, fmax=13)
# for i in range(len(mne_data)
# view = mne_data[i].compute_psd(method='multitaper', fmin=3, fmax=13)
#viei.plot()
#view = mne_data[20].compute_psd(method='multitaper', fmin=3, fmax=13) #o que está em mne_data é identificador; professor fez um for; frequência mínima  máxima
#print(len(mne_data))
#view.plot()
#print()

# como aplicar os filtros temporais no mne a partir dos dados do objeto epox?

In [ ]:
import warnings
warnings.filterwarnings('ignore')

for i in range(len(mne_data)):
    # view = mne_data.compute_psd(method='welch', fmin=3, fmax=13)
    view = mne_data[i].compute_psd(method='multitaper', fmin=3, fmax=13,
                                   verbose=False)
    view.plot(show=False)
    print()
    plt.title('Domínio da frequência')
    plt.axvline(x=float(list(mne_data[i].event_id)[0]), linestyle='--',
                color='green')
    plt.show()
print()